In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import *
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

import panel as pn
hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint
import utils as u

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

To deinf a curve on a plane (ie. planary curve) we need
- parameter, eg. p $\in [0,1]$
- two functions $x(p)$ and $y(p)$, which define the coordinate of the point $C(p)$ in x and y axis, respectively

In [ ]:
class Curve(param.Parameterized):
    p = param.Number(default=0.0, bounds=(0,1))
    
    def __init__(self, xfunc=None, yfunc=None):
        self.xfunc= xfunc or (lambda p: np.sin(2*np.pi*p))
        self.yfunc = yfunc or (lambda p: np.cos(2*np.pi*p))

        self.overview = self.get_overview()
        self.x = self.xfunc(self.p)
        self.y = self.yfunc(self.p)
        self.dmap_point = hv.Dynamic(hv.Points( [(self.x, self.y)], kdims=['x','y']))

        
    @param.depends('p')
    def update_xy(self):
        print('p changed and get_xy is called')
        self.x = self.xfunc(self.p)
        self.y = self.yfunc(self.p)
        self.hv_point = hv
        
    def get_overview(self):
        xs = [self.xfunc(p) for p in np.linspace(0,1,100)]
        ys = [self.yfunc(p) for p in np.linspace(0,1,100)]
        overview = hv.Points( (xs,ys), ['x','y'] ).opts(padding=0.1)
#         display(points)
        return overview
    
    def view(self):
        display(self.overview * hv.

In [ ]:
curve = Curve()

In [ ]:
curve.show()

In [ ]:
PStream = Stream.define('pstream', p=0.0, bounds=(0,1))

In [ ]:
pstream = PStream()

In [ ]:
pstream.print_param_values()

In [ ]:
pstream.event(p=2.)

In [ ]:
#using `pipe` to push data to the visualization
## 1. setup a dynamicmap with empty element 
pipe = Pipe(data=[])
dmap_points = hv.DynamicMap(hv.Points,streams=[pipe])
dmap_points.opts(
    opts.Points(color='green', 
                size=5,
                xlim=(-1,1), ylim=(-1,1),width=500, height=500)
).opts(padding=0.1);


## Curve Generator

In [ ]:
def gen_curve(cfunc, n_steps=100):
    p = 0
    dp = 1./n_steps
    count = 0
    while p<=1:
        x,y = cfunc(p)
        p, curr_p = p+dp, p
        count, curr_count = count+1, count
        yield pd.DataFrame([(curr_count, curr_p, x, y)], columns=['count', 'p', 'x','y'])
        

In [ ]:
n = 100
ps = np.linspace(0,1,num=n)

# define curve function
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)
cfunc = lambda p: (xfunc(p), yfunc(p))

In [ ]:
# another function
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.sin(2*np.pi*p)
cfunc = lambda p: (xfunc(p), yfunc(p))

In [ ]:
# another function
xfunc = lambda p: np.sin(4*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)
cfunc = lambda p: (xfunc(p), yfunc(p))

In [ ]:
cfunc(0), cfunc(1)

In [ ]:
g_curve = gen_curve(cfunc, n)
next(g_curve)
# 

---
## Method1: Buffer to push data
Push data  using pipe or buffer

In [ ]:
# Define buffer 
example = pd.DataFrame({'count': [], 'p': [], 'x':[], 'y':[]})
dfstream = Buffer(example, length=100, index=False)

In [ ]:
## visualization components
H,W = 500,500
dmap_points = hv.DynamicMap(
    lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
    streams=[dfstream]).opts(color='p')
curve_opts = opts.Points(size=5,xlim=(-1,1), ylim=(-1,1),width=W, height=H)
dmap_points.opts(curve_opts);
# dmap_points

In [ ]:
dmap_x = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
    streams=[dfstream]).opts(color='p')
xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                    padding=0.1, invert_axes=True, invert_yaxis=True)

dmap_x.opts(xopts);
# dmap_x

In [ ]:
dmap_y = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
    streams=[dfstream]).opts(color='p')
yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
dmap_y.opts(yopts);
# dmap_y


In [ ]:
(
    dmap_points + dmap_y
    + dmap_x
).cols(2)

In [ ]:
# send data through the buffer
dfstream.clear()
n = 100
g_curve = gen_curve(cfunc, n)
for i in range(n):
    dfstream.send(next(g_curve))
    time.sleep(0.3)

---
## Method2: Use `streamz.dataframe` and `hv.streams.Buffer`

In [ ]:
import streamz
import streamz.dataframe

In [ ]:
sdf = streamz.dataframe.DataFrame(example=example)

In [ ]:
sdf.example


In [ ]:
next(gen_curve())


In [ ]:
dmap2 = hv.DynamicMap(
    lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
    streams=[Buffer(sdf)]).opts(color='p')
# dmap2

In [ ]:
dmap2_x = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
    streams=[Buffer(sdf)]).opts(color='p')

xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                    padding=0.1, invert_axes=True, invert_yaxis=True)

dmap2_x.opts(xopts);
# dmap_x

In [ ]:
dmap2_y = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
    streams=[Buffer(sdf)]).opts(color='p')
yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1)
dmap_y.opts(yopts);

In [ ]:
g_curve = gen_curve()
for i in range(100):
    sdf.emit(next(g_curve))

In [ ]:
(dmap2 + dmap2_y +dmap2_x).cols(2)

## Let's encaptulate the simulation process, given a curve generator 


In [ ]:
def get_new_plots(dfstream):
    
    ## visualization components
    H,W = 500,500
    dmap_points = hv.DynamicMap(
        lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
        streams=[dfstream]).opts(color='p')
    
    curve_opts = opts.Points(size=5,xlim=(-1,1), ylim=(-1,1),width=W, height=H)
    dmap_points.opts(curve_opts);
    # dmap_points
    
    
    dmap_x = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
    streams=[dfstream]).opts(color='p')
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)

    dmap_x.opts(xopts);
    
    dmap_y = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
    streams=[dfstream]).opts(color='p')
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    dmap_y.opts(yopts);

    overlay = (dmap_points + dmap_y + dmap_x).cols(2)
    return overlay

### Setup for the simulation

In [ ]:
# Define buffer 
example = pd.DataFrame({'count': [], 'p': [], 'x':[], 'y':[]})
dfstream = Buffer(example, length=100, index=False)

# Define curve function
n_steps = 100
a,b = 1,0.5
xfunc = lambda p: (a-b)*np.cos(p) + b*np.cos((a/b-1)*p)
yfunc = lambda p: (a-b)*np.sin(p) - b*np.sin((a/b-1)*p)
cfunc = lambda p: (xfunc(p), yfunc(p))
g_curve = gen_curve(cfunc, n_steps)


# run
# get a new overlay
display(get_new_plots(dfstream))


# send data through the buffer
dfstream.clear()
for i in range(n_steps):
    dfstream.send(next(g_curve))
    time.sleep(0.3)


Try the simulation with various curve equations
- [fifty famous curves](https://elepa.files.wordpress.com/2013/11/fifty-famous-curves.pdf)

## Let's make a simulator class
- [important reference](http://holoviews.org/user_guide/Dashboards.html)
- [Linking pn.widgets and holoviews plots](https://panel.pyviz.org/user_guide/Links.html)

In [ ]:
import streamz
import streamz.dataframe

In [ ]:
class Simulator(param.Parameterized):
    
    ################################################################################
    # Instance Parameters
    ################################################################################
    a = param.Number(default=1.0)
    b = param.Number(default=0.25)
    to_run_clean = param.Boolean(False, doc='Simulation run boolean parameter')
    
    
    ################################################################################
    # Constant class properties
    ################################################################################
    H,W = 500,500
    curve_opts = opts.Points(size=5,xlim=(-1,1), ylim=(-1,1),width=W, height=H)
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)

    
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, n_steps=100, **kwargs):
        super().__init__(**kwargs)

        self.count = defaultdict(int)
        self.n_steps = n_steps
        self.example = pd.DataFrame({'count': [], 'p': [], 'x':[], 'y':[]})
        self.data_src = streamz.dataframe.DataFrame(example=self.example)
        self.dfstream = Buffer(self.data_src, length=100, index=False)
        self.set_dmap_curve()
        self.set_dmap_x()
        self.set_dmap_y()
        self.overlay = (self.dmap_curve + self.dmap_y + self.dmap_x).cols(2)
        
        self.set_funcs()

    def set_dmap_curve(self):
        self.count['set_curve'] += 1

        dmap_curve = hv.DynamicMap(
            lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
            streams=[self.dgittfstream]).opts(color='p')
        self.dmap_curve = dmap_curve.opts(self.curve_opts);

    def set_dmap_x(self):
        self.count['set_x'] += 1
        dmap_x = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_x = dmap_x.opts(self.xopts)
        
    def set_dmap_y(self):
        self.count['set_y'] += 1
        dmap_y = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_y = dmap_y.opts(self.yopts)
        
    @param.depends('a','b',watch=True)
    def set_funcs(self):
        print('set_funcs called')
        self.count['set_funcs'] += 1
        self.xfunc = lambda p: (self.a-self.b)*np.cos(p) + self.b*np.cos((self.a/self.b-1)*p)
        self.yfunc = lambda p: (self.a-self.b)*np.sin(p) - self.b*np.sin((self.a/self.b-1)*p)
        self.cfunc = lambda p: (self.xfunc(p), self.yfunc(p))
        
    ################################################################################
    # Holoviews components
    ################################################################################
    @param.depends('a','b','to_run_clean', watch=True)
    def get_count(self):
        df = pd.DataFrame(self.count.items(), columns=['name', 'count'])
        return hv.Table(df)
    

    @param.depends('to_run_clean', watch=True)
    def run_clean(self):
        # send data through the buffer
        self.count['run'] += 1
        self.dfstream.clear()

        if self.to_run_clean:
            curve_generator = gen_curve(self.cfunc, self.n_steps)

            for i in range(self.n_steps):
                self.data_src.emit(next(curve_generator))
                time.sleep(0.1)
        
        
    ################################################################################
    # Set DynamicMaps for linking the plots with parameters
    ################################################################################
    def view(self):
        return pn.panel(self.overlay)

    def dyn_count(self):
        return pn.panel(hv.DynamicMap(self.get_count))

    

In [ ]:
sim = Simulator()
# pn.panel( sim.view())

In [ ]:
pn.Column(
    pn.Row(pn.WidgetBox(sim.param), sim.view()),
    pn.panel(sim.dyn_count())
)

# Continue here!
Modified: Jul 13, 2019 (evening)


In [ ]:
# todo 1:
# Define a button widget (bokeh or panel.widgets object) outside here
# then, link this widget's parameter's callable to a parameterized object's method to set the value of p parameter to 0
# reset_button = Button(label='Reset')
# show(reset_button)

## but the widget shown by `show` function doesn't actually change the value. Check reset_button.clicks to confirm this.
## this is because we need to add `curdoc().add_root(bt)` and then launch bokeh server: `bokeh serve --show example.py`.
## but I dont want to use bokeh server....:( i just want simple interactivity.... what to do then?
## Ans: use param.Action with panel.widgets.Button (which is the default choice of widget for param.Action parameters

In [ ]:
class CurveSimulator(param.Parameterized):

    n_steps = param.Integer(label='Number of simulation steps', default=100)
    p = param.ObjectSelector(label='p', default=0., objects=np.linspace(0,1,num=n_steps.default))
    reset = param.Action(lambda x: x.reset_handler(), doc="Click to clear the buffer and reset p")
#     t_interval = param.Number(default=0.1)
    t_interval = param.Number(label='t_interval', doc='Time interval between plotting two points',
                              softbounds=(0., 5.),
                              default=0.2)

    
    ################################################################################
    # Constant class properties
    ################################################################################
    H,W = 500,500
    curve_opts = opts.Points(size=5,xlim=(-1,1), ylim=(-1,1),width=W, height=H)
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    
    
    ################################################################################
    # Parameter Dependencies
    ################################################################################    
    @param.depends('n_steps', watch=True)
    def _update_p(self):
        self.count['p'] += 1
        self.param['p'].objects = np.linspace(0,1,num=self.n_steps)
        print('updated p with new number of simulation steps: ', self.n_steps)
    
    @param.depends('p', watch=True)
    def send_point(self):
        point = pd.DataFrame([(self.p, *self.cfunc(self.p))], columns=['p', 'x','y'])
        self.data_src.emit(point)
        time.sleep(self.t_interval)
        
    def reset_handler(self):
        self.count['reset'] += 1
        self.set_param(p=0.0)
        self.dfstream.clear()

        
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, cfunc, n_steps=100, **kwargs):
        """
        Args:
        - cfunc (function): given an input of a float p in [0,1], returns (x,y), a 
        tuple of x and y coords
        
        - n_steps (int): number of simulation steps along the range of [0,1] for 
        the parameter, p
        """
        super().__init__(**kwargs) # this is super important
        self.count = defaultdict(int)
        self.cfunc = cfunc 
        self.n_steps = n_steps
        self.curve_generator = gen_curve(self.cfunc, self.n_steps)
        
        self.example = pd.DataFrame({'p': [], 'x':[], 'y':[]})
        self.data_src = streamz.dataframe.DataFrame(example=self.example)
        self.dfstream = Buffer(self.data_src, length=100, index=False)
        self.set_dmap_curve()
        self.set_dmap_x()
        self.set_dmap_y()
        self.overlay = (self.dmap_curve + self.dmap_y + self.dmap_x).cols(2)
    
    def set_curve_generator(self):
        self.curve_generator = gen_curve(self.cfunc, self.n_steps)

    def set_dmap_curve(self):
        self.count['set_curve'] += 1
        dmap_curve = hv.DynamicMap(
            lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_curve = dmap_curve.opts(self.curve_opts)
        
    def set_dmap_x(self):
        self.count['set_x'] += 1
        dmap_x = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_x = dmap_x.opts(self.xopts)
        
    def set_dmap_y(self):
        self.count['set_y'] += 1
        dmap_y = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_y = dmap_y.opts(self.yopts)
    
    
    ################################################################################
    # Display DynammicMaps
    ################################################################################ 
    def viewable(self):
        return self.overlay
        
    ################################################################################
    # Print Info
    ################################################################################ 
    def get_info(self):
        info = dict()
        info['n_steps']= self.n_steps
        info['example'] = self.example
        info['data_src'] = self.data_src
        info['dfstream'] = self.dfstream.data.describe() if len(self.dfstream.data)  else None
        info['to_go']= self.to_go
        return info 

    
        

In [ ]:
# define curve function
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)
cfunc = lambda p: (xfunc(p), yfunc(p))
c = CurveSimulator(cfunc)

In [ ]:
# pp(c.get_info())

In [ ]:

pn.Row(
    pn.Param(c.param, width=500, widgets={
        'p': pn.widgets.DiscretePlayer,
        'reset': pn.widgets.Button(name=c.param['reset'].label),
        't_interval': pn.widgets.FloatSlider
    }),
    pn.panel(c.viewable())
)


---
Modified: Jul 14, 2019
## Try different curve equations
- [resource](https://is.gd/3p4jBP)


In [ ]:
# Define curve function
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)
cfunc = lambda p: (xfunc(p), yfunc(p))
c = CurveSimulator(cfunc)

In [ ]:
# Create the widgets
pn.Row(
    pn.Param(c.param, widgets={'p': pn.widgets.DiscretePlayer,'reset': pn.widgets.Button}, width=500),
    pn.panel(c.viewable())
)

In [ ]:
def deg2rad(angle):
    return angle*np.pi/180.



class SpirographSimulator(CurveSimulator):

    R = param.Number(label='R', default=1., 
                     doc="""Radius of a circle (Doesn't affect the shape)""")
    k = param.Number(label='k', default=0.5, bounds=(0., 1.), softbounds=(1e-4, 1.0),
                     doc="""Ratio between the radius of the inner circle to the outer one""")
    l = param.Number(label='l', default=0.5, bounds=(0., 1.),
                     doc="""Parameter for how far the point A is located from the inner circle""")
    
    def __init__(self, **kwargs):
        self.temp_count = defaultdict(int)
        self._update_cfunc()
        super().__init__(self.cfunc, **kwargs)
        
    @param.depends('k','R','l', watch=True)
    def _update_cfunc(self):
        self.temp_count['cfunc'] += 1
        xfunc = lambda p: self.R*( (1-self.k)*np.cos(2*np.pi*p) + self.l*self.k*np.cos( ((1-self.k)/self.k)*2*np.pi*p))
        yfunc = lambda p: self.R*( (1-self.k)*np.sin(2*np.pi*p) + self.l*self.k*np.sin( ((1-self.k)/self.k)*2*np.pi*p))
        cfunc = lambda p: (xfunc(p), yfunc(p))
        self.cfunc = cfunc
        
        

In [ ]:
spiro = SpirographSimulator()

In [ ]:
# Create the widgets
pn.Row(
    pn.Param(spiro.param, width=500, widgets={
        'p': pn.widgets.DiscretePlayer,
        'reset': pn.widgets.Button,
        'k': pn.widgets.FloatSlider(name=spiro.param.k.label,
                                    value=spiro.k, 
                                    #callback_policy="mouseup" 
                                   ),
        'l': pn.widgets.FloatSlider(name=spiro.param.l.label,
                                    value=spiro.l, 
                                    #callback_policy="mouseup"
                                    ),
    }),
    pn.panel(spiro.viewable())
)

---
Modified: Jul 16, 2019
## PyTorch Version 
Incorporate autograd's auto-differentiation functionality to add velocity vector along the curve.
- We modify the original CurveSimulator to perform computations in torch-land (ie. using torch.Tensor and torch.nn.Function objects),
    and get the data points and the velocity at the current point on the curve.  
- Convert the tensor results to numpy and create a pd.DataFrame to send to the buffer stream

To do so, we need to modify two parts: 
- `cfunc` to use torch functions
- `send_point` method to convert the result of cfunc and velocity computation suitable for pd.DataFrame construction and buffer stream



In [ ]:
import torch
from torch import nn
from torchviz import make_dot, make_dot_from_trace

In [ ]:
xfunc = torch.sin
yfunc = torch.cos
cfunc = lambda p: (xfunc(p), yfunc(p))
tensor_p = torch.tensor(0.1, requires_grad=True)
make_dot(cfunc(tensor_p), params=dict(p=tensor_p))

In [ ]:
point = cfunc(tensor_p); point

# velocity vector = (dxfunc/dp (at tensor_p), dyfunc/dp at tensor_p)
tensor_p1 = torch.tensor(0.1, requires_grad=True)
tensor_p2 = torch.tensor(0.1, requires_grad=True)
tensor_x = xfunc(tensor_p1)
tensor_y = yfunc(tensor_p2)



In [ ]:
tensor_p1.grad.data.zero_()
# point.backward(torch.ones(point.shape))


In [ ]:
tensor_p.grad

In [ ]:
torch.stack(point)

In [ ]:
tensor_x.grad_fn

In [ ]:
tensor_x.backward()

In [ ]:
tensor_p1.grad

In [ ]:
tensor_y.backward()
print(tensor_p2.grad)

### Conversion helpers

In [ ]:
def euclidean2raidal(x,y):
    """
    Change the coordinate system from Euclidean to Radial.
    Returns (angle, magnitude), ie. (theta, r)
    """
    
    mag = np.sqrt(x**2+y**2)
    angle = (np.pi/2.) - np.arctan2(x/mag, y/mag)
    return(angle, mag)

            

In [ ]:
from holoviews import dim


### Simulator for curve and velocity

In [ ]:
class CurveVelocitySimulator(param.Parameterized):

    n_steps = param.Integer(label='Number of simulation steps', default=100)
    p = param.ObjectSelector(label='p', default=0., objects=np.linspace(0,1,num=n_steps.default))
    reset = param.Action(lambda x: x.reset_handler(), doc="Click to clear the buffer and reset p")
#     t_interval = param.Number(default=0.)
    t_interval = param.Number(label='t_interval', doc='Time interval between plotting two points',
                              softbounds=(0., 5.),
                              default=0.2)
    
    ################################################################################
    # Constant class properties
    ################################################################################
    H,W = 500,500
    curve_opts = opts.Points(size=5,width=W, height=H, padding=0.1, 
                            xlim=(-10,10), ylim=(-10,10),
                            color=dim('p')+10
                            )
    vel_opts = opts.VectorField(width=W, height=H, color='green', padding=0.1,
                                rescale_lengths=False,
                                xlim=(-10,10), ylim=(-10,10),
                               )#, bgcolor=(0,0,0,0))
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    
    
    ################################################################################
    # Parameter Dependencies
    ################################################################################    
    @param.depends('n_steps', watch=True)
    def _update_p(self):
        self.count['p'] += 1
        self.param['p'].objects = np.linspace(0,1,num=self.n_steps)
        print('updated p with new number of simulation steps: ', self.n_steps)
    
    @param.depends('p', watch=True)
    def send_point(self):
        x, y, dx, dy, ang, mag = self.get_point_and_vel(self.p)
        data = pd.DataFrame([(self.p, x,y,dx,dy,ang,mag)], columns=self.example.columns)
        self.data_src.emit(data)
        time.sleep(self.t_interval)
        
    def reset_handler(self):
        self.count['reset'] += 1
        self.set_param(p=0.0)
        self.dfstream.clear()

        
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, xfunc, yfunc, n_steps=100, **kwargs):
        """
        Args:
        - xfunc (torch.function): given an input of a float p in [0,1], returns x coord of the curve
        - yfunc (torch.function): given an input of a float p in [0,1], returns y coord of the curve

        
        - n_steps (int): number of simulation steps along the range of [0,1] for 
        the parameter, p
        """
        super().__init__(**kwargs) # this is super important
        self.count = defaultdict(int)
        self.xfunc, self.yfunc = xfunc, yfunc
        self.n_steps = n_steps
        
        self.example = pd.DataFrame({'p': [], 'x':[], 'y':[], 'dx':[], 'dy':[], 'ang':[], 'mag':[]})
        self.data_src = streamz.dataframe.DataFrame(example=self.example)
        self.dfstream = Buffer(self.data_src, length=100, index=False)
        self.set_dmap_curve()
        self.set_dmap_vel()
        self.set_dmap_x()
        self.set_dmap_y()
        self.overlay = (self.dmap_vel * self.dmap_curve + self.dmap_y + self.dmap_x).cols(2)
    

    def set_dmap_curve(self):
        self.count['set_curve'] += 1
        dmap_curve = hv.DynamicMap(
            lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
            streams=[self.dfstream])#.opts(color='p')
        self.dmap_curve = dmap_curve.opts(self.curve_opts)
        
    def set_dmap_vel(self):
        self.count['set_velocity'] += 1
        dmap_vel = hv.DynamicMap(
            lambda data: hv.VectorField(data, kdims=['x','y'], vdims=['ang','mag'] ),
            streams=[self.dfstream])#.opts(color='p')
        self.dmap_vel = dmap_vel.opts(self.vel_opts)
        
    def set_dmap_x(self):
        self.count['set_x'] += 1
        dmap_x = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_x = dmap_x.opts(self.xopts)
        
    def set_dmap_y(self):
        self.count['set_y'] += 1
        dmap_y = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_y = dmap_y.opts(self.yopts)
    
    
    ################################################################################
    # Display DynammicMaps
    ################################################################################ 
    def viewable(self):
        return self.overlay
        
    
    ################################################################################
    # Tensor computation helper
    ################################################################################  
    def get_point_and_vel(self,p):
        """
        Args: 
        - p (float): a float corresponding to a curve parameter's specific value. Must be in between [0,1]?

        Returns:
        - (x,y,dx,dy): a tuple of floats corresponding to xcoord, ycoord, x and y components of the velocity at (x,y) 
        """

        tensor_p1, tensor_p2 = torch.tensor([p,p]).unbind()
        tensor_p1.requires_grad_(); tensor_p2.requires_grad_()
        
        tensor_x, tensor_y = self.xfunc(tensor_p1),  self.yfunc(tensor_p2)

        # compute gradient
        tensor_x.backward()
        tensor_y.backward()
        dx = tensor_p1.grad.data.item()
        dy = tensor_p2.grad.data.item()
        ang,vel = euclidean2raidal(dx,dy)

        return (tensor_x.data.item(), tensor_y.data.item(), dx, dy, ang, vel)
            

In [ ]:
xfunc = lambda p: torch.log(torch.sin(p*2*np.pi))
yfunc = lambda p: torch.cos(p*2*np.pi)

c = CurveVelocitySimulator(xfunc, yfunc, 100)
c

In [ ]:
%%opts Curve [active_tools=['wheel_zoom']]
pn.Row(
    pn.Param(c.param, width=500, widgets={
        'p': pn.widgets.DiscretePlayer,
        'reset': pn.widgets.Button(name=c.param['reset'].label),
        't_interval': pn.widgets.FloatSlider
    }),
    pn.panel(c.viewable())
#     pn.Column(
#     pn.panel(c.dmap_vel), pn.panel(c.dmap_curve)
#     )

)


In [ ]:
hv.

In [ ]:
c.dmap_vel

In [ ]:
todo:
    - curve generator by creating the dataset all at once
    autograddd!!!:DDDD

---
Modified: Jul 16, 2019
## Use torch's autograd to compute velocity vector

In [ ]:
def get_point_and_vel(xfunc, yfunc, p):
    """
    Args: 
    - xfunc: torch function that takes in a tensor and outputs a scalar tensor correpsonding to the x-coordinate of the curve
    - yfunc: torch function that takes in a tensor and outputs a scalar tensor correpsonding to the y-coordinate of the curve
    - p (float or a list of floats): each float corresponding to a curve parameter's specific value. Must be in between [0,1]?
    
    Returns:
    
    - (x,y,dx,dy): a tuple of floats corresponding to xcoord, ycoord, x and y components of the velocity at (x,y) 
    each x,y,dx,dy has the same shape as p.
    In other words, this is a vectorized operation.
    """
    
    tensor_p1 = torch.tensor(p, requires_grad=True)
    tensor_p2 = torch.tensor(p, requires_grad=True)
    tensor_x, tensor_y =xfunc(tensor_p1), yfunc(tensor_p2)

    # compute gradient
    tensor_x.backward(torch.ones(tensor_x.shape))
    tensor_y.backward(torch.ones(tensor_y.shape))
    dx = tensor_p1.grad.data.numpy()
    dy = tensor_p2.grad.data.numpy()
    ang,mag = euclidean2raidal(dx,dy)

    return (tensor_x.data.numpy(), tensor_y.data.numpy(), dx, dy, ang, mag)

In [ ]:
def test_get_point_and_vel_single_p():
    p = np.pi
    xfunc = torch.sin
    yfunc = torch.cos
    x,y,dx,dy,ang,mag = get_point_and_vel(xfunc, yfunc, p)
    df =  pd.DataFrame([(x,y,dx,dy,ang,mag)], columns='x,y,dx,dy,ang,mag'.split(','))
    display(df.head())
   

In [ ]:
 
def test_get_point_and_vel_multiple_ps():
    p = np.linspace(0., 1., num=10, dtype=np.float32)
    xfunc = torch.sin
    yfunc = torch.cos
    x,y,dx,dy,ang,mag = get_point_and_vel(xfunc, yfunc, p)
    df =  pd.DataFrame(list(zip(x,y,dx,dy,ang,mag)), columns='x,y,dx,dy,ang,mag'.split(','))
    display(df.head())
# test_get_point_and_vel()

In [ ]:
def create_curve_data(xfunc, yfunc, n_steps=100):
    ps = np.linspace(0., 1., num=n_steps, dtype=np.float32)
    x,y,dx,dy,ang,mag = get_point_and_vel(xfunc,yfunc, ps)
    df =  pd.DataFrame(list(zip(x,y,dx,dy,ang,mag)), columns='x,y,dx,dy,ang,mag'.split(','))
    return df

In [ ]:
create_curve_data(xfunc, yfunc,10)

In [ ]:
class CurveBulkSimulator(param.Parameterized):
    """
    Creats all `n_steps` number of points on the curve at once 
    and saves in memory
    
    For simulation, each data point is emitted to the buffer stream 
    by reading from the in-memory data points
    
    Params:
    - n_steps: Number of points on the curve, ie. number of steps in [0,1] range
    - p : parameter for curve function. It runs from 0 to 1, inclusively
    - reset : action parameter that sets the p parameter to zero and clears out the buffer stream
    - t_interval: time interval to pause in between sending data points while simulation is running
    
    """
    

    n_steps = param.Integer(label='Number of simulation steps', default=100)
    p = param.ObjectSelector(label='p', default=0., objects=np.linspace(0.,1.,num=n_steps.default))
    pidx = param.ObjectSelector(label='pidx', default=0, objects=list(range(n_steps.default)))

    reset = param.Action(lambda x: x.reset_handler(), doc="Click to clear the buffer and reset p")
    t_interval = param.Number(label='t_interval', doc='Time interval between plotting two points',
                              softbounds=(0., 5.),
                              default=0.2)
    
    ################################################################################
    # Constant class properties
    ################################################################################
    H,W = 500,500
    curve_opts = opts.Points(size=5,width=W, height=H, padding=0.1, 
#                             xlim=(-10,10), ylim=(-10,10),
                            color=dim('p')*256-50
                            )
    vel_opts = opts.VectorField(width=W, height=H, color='green', padding=0.1,
#                                 rescale_lengths=False,
#                                 xlim=(-10,10), ylim=(-10,10),
                               )#, bgcolor=(0,0,0,0))
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    
    
    ################################################################################
    # Parameter Dependencies
    ################################################################################    
    @param.depends('n_steps', watch=True)
    def _update_p(self):
        self.count['p'] += 1
        self.param['p'].objects = np.linspace(0.,1.,num=self.n_steps)
        self.param['pidx'].objects = list(range(self.n_steps))
        print('updated p with new number of simulation steps: ', self.n_steps)
    
    @param.depends('n_steps', watch=True)
    def _update_curve_data(self):
        ps = np.linspace(0., 1., num=self.n_steps, dtype=np.float32)
        x,y,dx,dy,ang,mag = get_point_and_vel(self.xfunc, self.yfunc, ps)
        self.data = pd.DataFrame(list(zip(ps,x,y,dx,dy,ang,mag)), 
                                 columns='p,x,y,dx,dy,ang,mag'.split(','),
                                 dtype=np.float32)#.set_index('p')
        
    
    @param.depends('pidx', watch=True)
    def send_point(self):
        # p(float in [0,1] to index into np.linspace(0,1,n_steps) list
#         pidx = val2idx(p, 
        data = self.data.iloc[[self.pidx]]
        self.data_src.emit(data)
        time.sleep(self.t_interval)
    
    
    def reset_handler(self):
        self.count['reset'] += 1
        self.set_param(p=0.0)
        self.dfstream.clear()

        
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, xfunc, yfunc, n_steps=100, **kwargs):
        """
        Args:
        - xfunc (torch.function): given an input of a float p in [0,1], returns x coord of the curve
        - yfunc (torch.function): given an input of a float p in [0,1], returns y coord of the curve

        
        - n_steps (int): number of simulation steps along the range of [0,1] for 
        the parameter, p
        """
        super().__init__(**kwargs) # this is super important
        self.count = defaultdict(int)
        self.xfunc, self.yfunc = xfunc, yfunc
        self.n_steps = n_steps
        
        self.example = pd.DataFrame({'p': [], 'x':[], 'y':[], 'dx':[], 'dy':[], 'ang':[], 'mag':[]})
        self.data_src = streamz.dataframe.DataFrame(example=self.example)
        self.dfstream = Buffer(self.data_src, length=100, index=False)
        self.set_dmap_curve()
        self.set_dmap_vel()
        self.set_dmap_x()
        self.set_dmap_y()
        self.overlay = (self.dmap_vel * self.dmap_curve + self.dmap_y + self.dmap_x).cols(2)
    
        # create all curve data
        self._update_curve_data()
        
    def set_dmap_curve(self):
        self.count['set_curve'] += 1
        dmap_curve = hv.DynamicMap(
            lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
            streams=[self.dfstream])#.opts(color='p')
        self.dmap_curve = dmap_curve.opts(self.curve_opts)
        
    def set_dmap_vel(self):
        self.count['set_velocity'] += 1
        dmap_vel = hv.DynamicMap(
            lambda data: hv.VectorField(data, kdims=['x','y'], vdims=['ang','mag'] ),
            streams=[self.dfstream])#.opts(color='p')
        self.dmap_vel = dmap_vel.opts(self.vel_opts)
        
    def set_dmap_x(self):
        self.count['set_x'] += 1
        dmap_x = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_x = dmap_x.opts(self.xopts)
        
    def set_dmap_y(self):
        self.count['set_y'] += 1
        dmap_y = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_y = dmap_y.opts(self.yopts)
    
    
    ################################################################################
    # Display DynammicMaps
    ################################################################################ 
    def viewable(self):
        return self.overlay
        
    

In [ ]:
xfunc = lambda p: torch.cos(p*2*np.pi)* torch.sin(p*2*np.pi)
yfunc = lambda p: torch.cos(p*2*np.pi)

c = CurveBulkSimulator(xfunc, yfunc)

In [ ]:
c.data.head()

In [ ]:
%%opts Points [active_tools=['wheel_zoom'], tools=['hover']]
pn.Column(
    pn.Param(c.param.n_steps),
    pn.Param(c.param.pidx, widgets={'pidx': pn.widgets.DiscretePlayer}),
    pn.Param(c.param.reset, widgets={'reset': pn.widgets.Button(name=c.param['reset'].label)}),
    pn.Param(c.param.t_interval, widgets={'t_interval': pn.widgets.FloatSlider}),
    pn.panel(c.viewable())
# #     pn.Column(
# #     pn.panel(c.dmap_vel), pn.panel(c.dmap_curve)
# #     )
 )


---
Modified: Jul 16, 2019
## Use torch's autograd to compute velocity vector

In [ ]:
def get_point_and_vel(xfunc, yfunc, p):
    """
    Args: 
    - xfunc: torch function that takes in a tensor and outputs a scalar tensor correpsonding to the x-coordinate of the curve
    - yfunc: torch function that takes in a tensor and outputs a scalar tensor correpsonding to the y-coordinate of the curve
    - p (float): a float corresponding to a curve parameter's specific value. Must be in between [0,1]?
    
    Returns:
    - (x,y,dx,dy): a tuple of floats corresponding to xcoord, ycoord, x and y components of the velocity at (x,y) 
    """
    
    tensor_p1, tensor_p2 = torch.tensor([p,p]).unbind()
    tensor_p1.requires_grad_()
    tensor_p2.requires_grad_()
    tensor_x, tensor_y =xfunc(tensor_p1), yfunc(tensor_p2)

    # compute gradient
    tensor_x.backward()
    tensor_y.backward()
    dx = tensor_p1.grad.data.item()
    dy = tensor_p2.grad.data.item()
    ang,mag = euclidean2raidal(dx,dy)

    return (tensor_x.data.item(), tensor_y.data.item(), dx, dy, ang, mag)

In [ ]:
def test_get_point_and_vel():
    p = np.pi
    xfunc = torch.sin
    yfunc = torch.cos
    x,y,dx,dy,ang,mag = get_point_and_vel(xfunc, yfunc, p)
    df =  pd.DataFrame([(x,y,dx,dy,ang,mag)], columns='x,y,dx,dy,ang,mag'.split(','))
    print(df)
    

In [ ]:
p = np.pi
xfunc = torch.sin
yfunc = torch.cos
print(get_point_and_vel(xfunc, yfunc, p))

In [ ]:
x,y,dx,dy,ang,mag

In [ ]:
temp = pd.DataFrame([(x,y,dx,dy,ang,mag)], columns='x,y,dx,dy,ang,mag'.split(','))

In [ ]:
temp

In [ ]:
hv.VectorField(temp['x,y,ang,mag'.split(',')])

In [ ]:
temp['x,y,ang,mag'.split(',')]

In [ ]:
from holoviews import dim


In [ ]:
vfield_opts = opts.VectorField(width=500, height=500, color='blue',
                               magnitude=dim('mag').norm()*0.5, 
                               rescale_lengths=False)

In [ ]:
vfield = hv.VectorField( [
    (0, 0, np.pi, 0.1),
    (0, 1,np.pi/4., 0.5),
    (0, 1,np.pi, 0.2)
], kdims=['x','y'], vdims=['ang','mag']
)#.opts(width=500, height=500,color='blue',magnitude=dim('Magnitude').norm()*0.2, rescale_lengths=False)

In [ ]:
vfield.opts(vfield_opts)

In [ ]:
data = [(0.,1., np.pi, 10.0), (0.,0., np.pi, 20.0), (0.,-1., np.pi, 20.0), (-1.,-1., np.pi, 20.0)] 
df = pd.DataFrame(data, columns='x,y,ang,mag'.split(','))
df

In [ ]:
vfield = hv.VectorField( df, kdims=['x','y'], vdims=['ang','mag'] )
# vfield.opts(opts.VectorField(magnitude=dim('mag').min(),rescale_lengths=False))

In [ ]:
vfield.opts.clear()

In [ ]:
vfield.opts(opts.VectorField(rescale_lengths=True))

### Todo2: levelset implementation
- [ ] how to implement affine transformation in discrete space?
    - we need a `warp` function